In [1]:
!nvidia-smi

Fri Feb  4 16:06:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 匯入函式庫

In [2]:
!pip install openpyxl==3.0.9

In [3]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


# CPU/GPU、自定義資料集、模型、訓練函數

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = torch.from_numpy(X)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [7]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [8]:
class Self_Attention(nn.Module):
    def __init__(self, num_layers, hidden_size, length_input_sequence):
        super().__init__()
        self.num_layers = num_layers
        
        self.Input_First_HiddenLayer = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1)
        self.Query_Key_Value_1 = nn.ModuleList([nn.Linear(1, hidden_size), nn.Linear(1, hidden_size), nn.Linear(1, hidden_size)])

        self.Second_And_Following_HiddenLayer = nn.ModuleList([nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1) for i in range(num_layers - 1)])
        self.List_of_Query_Key_Value = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_size, hidden_size) for q_k_v in range(3)]) for i in range(num_layers - 1)])

        self.OutputLayer = nn.Linear(hidden_size, 1)

        # generate mask
        mask = (torch.triu(torch.ones(length_input_sequence, length_input_sequence)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        self.register_buffer('mask', mask)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        Query1 = self.Query_Key_Value_1[0](input)
        Key1 = self.Query_Key_Value_1[1](input)
        Value1 = self.Query_Key_Value_1[2](input)
        hidden, _ = self.Input_First_HiddenLayer(Query1, Key1, Value1, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        if self.num_layers > 1:
            for i, Second_And_Following_HiddenLayer in enumerate(self.Second_And_Following_HiddenLayer):
                Query = self.List_of_Query_Key_Value[i][0](hidden)
                Key = self.List_of_Query_Key_Value[i][1](hidden)
                Value = self.List_of_Query_Key_Value[i][2](hidden)
                hidden, _ = Second_And_Following_HiddenLayer(Query, Key, Value, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [9]:
def train_under_config(model_type,
                       forex_data,
                       length_input_sequence,
                       num_epochs,
                       num_hidden_layers,
                       num_hidden_sizes,
                       batch_sizes,
                       device):
    '''
    model_type: LSTM or Self_Attention,
    forex_data,
    length_input_sequence,
    num_epochs,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    # dataset
    training_data = forex_data.loc['1981-01-01':'2016-12-31'] # training_data = training set + validation set
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataloader
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # model
    if model_type == Self_Attention:
        model = model_type(num_hidden_layers, num_hidden_sizes, length_input_sequence).double()
    else:
        model = model_type(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    # training
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
    
    return model

In [10]:
# residue pred
def ypred(data, length_input_sequence, model, device):
    '''
    data,
    length_input_sequence,
    model,
    device
    '''
    # dataset
    testing_start_index = len(data.loc['1981-01-01':'2016-12-31']) - length_input_sequence
    testing_end_index = len(data.loc['1981-01-01':'2020-12-31'])
    testing_data = data[testing_start_index:testing_end_index]
    testing_dataset = TimeSeriesDataset(testing_data, length_input_sequence)
    # dataloader
    data_len = len(testing_dataset)
    testing_dataloader = DataLoader(testing_dataset, batch_size=data_len, shuffle=False)
    # evaluating
    model.to(device)
    model.eval()
    for X, y in  testing_dataloader:
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            ypred = model(X)
            ypred = np.array(ypred.cpu()).flatten()
    return ypred

In [11]:
def prediction_on_testset(model_type, linear_data, residue_data, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs):
    model = train_under_config(model_type,
                               residue_data,
                               length_input_sequence,
                               num_epochs,
                               num_hidden_layers,
                               num_hidden_sizes,
                               batch_sizes,
                               device)
    model_type_linear_pred = np.array(linear_data.loc['2017-01-01':'2020-12-31'])
    model_type_residue_pred = ypred(residue_data, length_input_sequence, model, device)
    model_type_pred = model_type_linear_pred + model_type_residue_pred
    return model_type_pred

In [12]:
# random walk prediction
def rw_pred(testing_data):
    return np.zeros(len(testing_data))

In [13]:
def root_mean_squared_error(ypred, ytrue):
    squared_error = (ypred - ytrue) ** 2
    mean_squared_error = sum(squared_error) / len(squared_error)
    return np.sqrt(mean_squared_error)

In [14]:
def mean_absolute_error(ypred, ytrue):
    absolute_error = abs(ypred - ytrue)
    return sum(absolute_error) / len(absolute_error)

In [15]:
def direction_accuracy(ypred, ytrue):
    ypred_elementwise_dot_ytrue = ypred * ytrue
    same_direction_rate = sum(ypred_elementwise_dot_ytrue > 0) / len(ypred_elementwise_dot_ytrue)
    return same_direction_rate

# 匯入訓練資料（訓練加驗證集）訓練，並產生模型預測值

## Recursive window

In [16]:
recursive_linear = pd.read_csv('./recursive_linear_prediction.csv', index_col=0)
recursive_linear = recursive_linear.astype('float64')
recursive_linear.index = pd.to_datetime(recursive_linear.index)

cad_recursive_linear = recursive_linear.iloc[:,0]
aud_recursive_linear = recursive_linear.iloc[:,1]
gbp_recursive_linear = recursive_linear.iloc[:,2]

recursive_linear

,cad_recursive_linear_prediction,aud_recursive_linear_prediction,gbp_recursive_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000012,-0.000047,0.000109
2020-12-28,-0.000012,-0.000047,-0.000048
2020-12-29,-0.000012,-0.000048,-0.000323
2020-12-30,-0.000012,-0.000047,0.000074


In [17]:
recursive_residue = pd.read_csv('./recursive_linear_residue.csv', index_col=0)
recursive_residue = recursive_residue.astype('float64')
recursive_residue.index = pd.to_datetime(recursive_residue.index)

cad_recursive_residue = recursive_residue.iloc[:,0]
aud_recursive_residue = recursive_residue.iloc[:,1]
gbp_recursive_residue = recursive_residue.iloc[:,2]

recursive_residue

,cad_recursive_linear_residue,aud_recursive_linear_residue,gbp_recursive_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000117,0.001100,0.000038
2020-12-28,0.001426,-0.003377,-0.007949
2020-12-29,0.002194,0.003604,0.004107
2020-12-30,0.005254,0.010510,0.008920


### LSTM

In [18]:
model_type = LSTM

#### cad

In [19]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [20]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [21]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [22]:
cad_recursive_LSTM_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [23]:
aud_recursive_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000047
2020-12-28   -0.000047
2020-12-29   -0.000048
2020-12-30   -0.000047
2020-12-31   -0.000046
Name: aud_recursive_linear_prediction, Length: 10435, dtype: float64

In [24]:
aud_recursive_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001100
2020-12-28   -0.003377
2020-12-29    0.003604
2020-12-30    0.010510
2020-12-31    0.001087
Name: aud_recursive_linear_residue, Length: 10435, dtype: float64

In [25]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 3
num_hidden_sizes = 1600
batch_sizes = 128
num_epochs = 15

In [26]:
aud_recursive_LSTM_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [27]:
gbp_recursive_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000109
2020-12-28   -0.000048
2020-12-29   -0.000323
2020-12-30    0.000074
2020-12-31    0.000251
Name: gbp_recursive_linear_prediction, Length: 10435, dtype: float64

In [28]:
gbp_recursive_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25    0.000038
2020-12-28   -0.007949
2020-12-29    0.004107
2020-12-30    0.008920
2020-12-31    0.003047
Name: gbp_recursive_linear_residue, Length: 10435, dtype: float64

In [29]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 25
batch_sizes = 256
num_epochs = 5

In [30]:
gbp_recursive_LSTM_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [31]:
model_type = Self_Attention

#### cad

In [32]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [33]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [34]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 15

In [35]:
cad_recursive_Self_Attention_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [36]:
aud_recursive_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000047
2020-12-28   -0.000047
2020-12-29   -0.000048
2020-12-30   -0.000047
2020-12-31   -0.000046
Name: aud_recursive_linear_prediction, Length: 10435, dtype: float64

In [37]:
aud_recursive_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001100
2020-12-28   -0.003377
2020-12-29    0.003604
2020-12-30    0.010510
2020-12-31    0.001087
Name: aud_recursive_linear_residue, Length: 10435, dtype: float64

In [38]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 1
num_hidden_sizes = 800
batch_sizes = 16
num_epochs = 30

In [39]:
aud_recursive_Self_Attention_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [40]:
gbp_recursive_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000109
2020-12-28   -0.000048
2020-12-29   -0.000323
2020-12-30    0.000074
2020-12-31    0.000251
Name: gbp_recursive_linear_prediction, Length: 10435, dtype: float64

In [41]:
gbp_recursive_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25    0.000038
2020-12-28   -0.007949
2020-12-29    0.004107
2020-12-30    0.008920
2020-12-31    0.003047
Name: gbp_recursive_linear_residue, Length: 10435, dtype: float64

In [42]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 20

In [43]:
gbp_recursive_Self_Attention_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

## Rolling window

In [44]:
rolling_linear = pd.read_csv('./rolling_linear_prediction.csv', index_col=0)
rolling_linear = rolling_linear.astype('float64')
rolling_linear.index = pd.to_datetime(rolling_linear.index)

cad_rolling_linear = rolling_linear.iloc[:,0]
aud_rolling_linear = rolling_linear.iloc[:,1]
gbp_rolling_linear = rolling_linear.iloc[:,2]

rolling_linear

,cad_rolling_linear_prediction,aud_rolling_linear_prediction,gbp_rolling_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000002,-0.000015,0.000171
2020-12-28,-0.000002,-0.000014,0.000020
2020-12-29,-0.000002,-0.000014,-0.000245
2020-12-30,-0.000002,-0.000014,0.000138


In [45]:
rolling_residue = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
rolling_residue = rolling_residue.astype('float64')
rolling_residue.index = pd.to_datetime(rolling_residue.index)

cad_rolling_residue = rolling_residue.iloc[:,0]
aud_rolling_residue = rolling_residue.iloc[:,1]
gbp_rolling_residue = rolling_residue.iloc[:,2]

rolling_residue

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


### LSTM

In [46]:
model_type = LSTM

#### cad

In [47]:
cad_rolling_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000002
2020-12-28   -0.000002
2020-12-29   -0.000002
2020-12-30   -0.000002
2020-12-31   -0.000001
Name: cad_rolling_linear_prediction, Length: 10435, dtype: float64

In [48]:
cad_rolling_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 10435, dtype: float64

In [49]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [50]:
cad_rolling_LSTM_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [51]:
aud_rolling_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000015
2020-12-28   -0.000014
2020-12-29   -0.000014
2020-12-30   -0.000014
2020-12-31   -0.000013
Name: aud_rolling_linear_prediction, Length: 10435, dtype: float64

In [52]:
aud_rolling_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 10435, dtype: float64

In [53]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 256
num_epochs = 25

In [54]:
aud_rolling_LSTM_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [55]:
gbp_rolling_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000171
2020-12-28    0.000020
2020-12-29   -0.000245
2020-12-30    0.000138
2020-12-31    0.000309
Name: gbp_rolling_linear_prediction, Length: 10435, dtype: float64

In [56]:
gbp_rolling_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 10435, dtype: float64

In [57]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 25
batch_sizes = 256
num_epochs = 5

In [58]:
gbp_rolling_LSTM_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [59]:
model_type = Self_Attention

#### cad

In [60]:
cad_rolling_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000002
2020-12-28   -0.000002
2020-12-29   -0.000002
2020-12-30   -0.000002
2020-12-31   -0.000001
Name: cad_rolling_linear_prediction, Length: 10435, dtype: float64

In [61]:
cad_rolling_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 10435, dtype: float64

In [62]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 15

In [63]:
cad_rolling_Self_Attention_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [64]:
aud_rolling_linear

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000015
2020-12-28   -0.000014
2020-12-29   -0.000014
2020-12-30   -0.000014
2020-12-31   -0.000013
Name: aud_rolling_linear_prediction, Length: 10435, dtype: float64

In [65]:
aud_rolling_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 10435, dtype: float64

In [66]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 1
num_hidden_sizes = 800
batch_sizes = 16
num_epochs = 30

In [67]:
aud_rolling_Self_Attention_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [68]:
gbp_rolling_linear

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000171
2020-12-28    0.000020
2020-12-29   -0.000245
2020-12-30    0.000138
2020-12-31    0.000309
Name: gbp_rolling_linear_prediction, Length: 10435, dtype: float64

In [69]:
gbp_rolling_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 10435, dtype: float64

In [70]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 20

In [71]:
gbp_rolling_Self_Attention_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

# 輸出模型預測與比較

## 輸出模型預測

In [72]:
LSTM_recursive_prediction = pd.DataFrame(np.column_stack((cad_recursive_LSTM_pred, aud_recursive_LSTM_pred, gbp_recursive_LSTM_pred)))
LSTM_recursive_prediction.columns = ['cad_recursive_LSTM_pred', 'aud_recursive_LSTM_pred', 'gbp_recursive_LSTM_pred']
LSTM_recursive_prediction.to_csv('./LSTM_recursive_prediction.csv')
LSTM_recursive_prediction

,cad_recursive_LSTM_pred,aud_recursive_LSTM_pred,gbp_recursive_LSTM_pred
0,-0.000065,0.000897,-0.000205
1,0.000016,0.000791,-0.000549
2,-0.000044,0.000818,-0.000460
3,-0.000117,0.000821,-0.000176
4,-0.000008,0.000760,-0.000203
...,...,...,...
1039,0.000027,0.000976,-0.000232
1040,-0.000058,0.000683,-0.000402
1041,-0.000027,0.000869,-0.000611
1042,-0.000045,0.000745,-0.000250


In [73]:
LSTM_rolling_prediction = pd.DataFrame(np.column_stack((cad_rolling_LSTM_pred, aud_rolling_LSTM_pred, gbp_rolling_LSTM_pred)))
LSTM_rolling_prediction.columns = ['cad_rolling_LSTM_pred', 'aud_rolling_LSTM_pred', 'gbp_rolling_LSTM_pred']
LSTM_rolling_prediction.to_csv('./LSTM_rolling_prediction.csv')
LSTM_rolling_prediction

,cad_rolling_LSTM_pred,aud_rolling_LSTM_pred,gbp_rolling_LSTM_pred
0,-0.000065,0.001197,-0.000205
1,0.000016,0.001142,-0.000549
2,-0.000045,0.001312,-0.000462
3,-0.000117,0.001169,-0.000177
4,-0.000008,0.001096,-0.000204
...,...,...,...
1039,0.000036,0.001275,-0.000168
1040,-0.000048,0.001076,-0.000332
1041,-0.000017,0.001384,-0.000531
1042,-0.000035,0.001236,-0.000184


In [74]:
Self_Attention_recursive_prediction = pd.DataFrame(np.column_stack((cad_recursive_Self_Attention_pred, aud_recursive_Self_Attention_pred, gbp_recursive_Self_Attention_pred)))
Self_Attention_recursive_prediction.columns = ['cad_recursive_Self_Attention_pred', 'aud_recursive_Self_Attention_pred', 'gbp_recursive_Self_Attention_pred']
Self_Attention_recursive_prediction.to_csv('./Self_Attention_recursive_prediction.csv')
Self_Attention_recursive_prediction

,cad_recursive_Self_Attention_pred,aud_recursive_Self_Attention_pred,gbp_recursive_Self_Attention_pred
0,0.000123,-0.001629,-0.000229
1,0.000154,-0.001620,-0.000563
2,0.000076,-0.001619,-0.000485
3,-0.000028,-0.001618,-0.000241
4,-0.000047,-0.001618,-0.000258
...,...,...,...
1039,0.000366,-0.001620,-0.000203
1040,0.000302,-0.001620,-0.000405
1041,0.000212,-0.001621,-0.000644
1042,0.000135,-0.001620,-0.000358


In [75]:
Self_Attention_rolling_prediction = pd.DataFrame(np.column_stack((cad_rolling_Self_Attention_pred, aud_rolling_Self_Attention_pred, gbp_rolling_Self_Attention_pred)))
Self_Attention_rolling_prediction.columns = ['cad_rolling_Self_Attention_pred', 'aud_rolling_Self_Attention_pred', 'gbp_rolling_Self_Attention_pred']
Self_Attention_rolling_prediction.to_csv('./Self_Attention_rolling_prediction.csv')
Self_Attention_rolling_prediction

,cad_rolling_Self_Attention_pred,aud_rolling_Self_Attention_pred,gbp_rolling_Self_Attention_pred
0,0.000123,-0.001629,-0.000229
1,0.000154,-0.001620,-0.000563
2,0.000076,-0.001620,-0.000486
3,-0.000029,-0.001619,-0.000242
4,-0.000048,-0.001618,-0.000259
...,...,...,...
1039,0.000376,-0.001588,-0.000137
1040,0.000313,-0.001588,-0.000334
1041,0.000223,-0.001588,-0.000563
1042,0.000146,-0.001587,-0.000290


## 載入實際匯率變動率（取對數後一階差分）

In [76]:
groundtruth = pd.read_excel('./CAD_AUD_GBP.xlsx', index_col=0, skiprows=3)
groundtruth = groundtruth.iloc[1:,:]
groundtruth = groundtruth.astype('float64')
groundtruth.index = pd.to_datetime(groundtruth.index)
groundtruth = np.log(groundtruth)
groundtruth = groundtruth.diff().dropna()

groundtruth = groundtruth.loc['2017-01-01':'2020-12-31']
cad_groundtruth = groundtruth.iloc[:,0]
aud_groundtruth = groundtruth.iloc[:,1]
gbp_groundtruth = groundtruth.iloc[:,2]

groundtruth

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
2017-01-02,-0.000269,-0.003335,-0.005037
2017-01-03,0.001209,0.004860,-0.003345
2017-01-04,0.009488,0.008826,0.007003
2017-01-05,0.005703,0.007523,0.007760
2017-01-06,-0.000926,-0.005055,-0.010686
...,...,...,...
2020-12-25,-0.000129,0.001052,0.000148
2020-12-28,0.001414,-0.003425,-0.007996
2020-12-29,0.002182,0.003556,0.003784
2020-12-30,0.005242,0.010463,0.008994


## 模型比較（計算模型的 RMSE, MAE, DA）

### Random walk pred

In [77]:
random_walk_pred = rw_pred(cad_groundtruth)

cad_random_walk_RMSE = root_mean_squared_error(random_walk_pred, cad_groundtruth)
cad_random_walk_MAE = mean_absolute_error(random_walk_pred, cad_groundtruth)
cad_random_walk_DA = 0.5
aud_random_walk_RMSE = root_mean_squared_error(random_walk_pred, aud_groundtruth)
aud_random_walk_MAE = mean_absolute_error(random_walk_pred, aud_groundtruth)
aud_random_walk_DA = 0.5
gbp_random_walk_RMSE = root_mean_squared_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_MAE = mean_absolute_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_DA = 0.5

In [78]:
cad_random_walk_RMSE, cad_random_walk_MAE, cad_random_walk_DA

(0.004365277893580916, 0.0032165540648211085, 0.5)

In [79]:
aud_random_walk_RMSE, aud_random_walk_MAE, aud_random_walk_DA

(0.005655793284191722, 0.00416259953667118, 0.5)

In [80]:
gbp_random_walk_RMSE, gbp_random_walk_MAE, gbp_random_walk_DA

(0.0056882604301849125, 0.004202877474568311, 0.5)

### Recursive LSTM pred

In [81]:
cad_recursive_LSTM_RMSE = root_mean_squared_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_MAE = mean_absolute_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_DA = direction_accuracy(cad_recursive_LSTM_pred, cad_groundtruth)
aud_recursive_LSTM_RMSE = root_mean_squared_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_MAE = mean_absolute_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_DA = direction_accuracy(aud_recursive_LSTM_pred, aud_groundtruth)
gbp_recursive_LSTM_RMSE = root_mean_squared_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_MAE = mean_absolute_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_DA = direction_accuracy(gbp_recursive_LSTM_pred, gbp_groundtruth)

In [82]:
cad_recursive_LSTM_RMSE, cad_recursive_LSTM_MAE, cad_recursive_LSTM_DA

(0.004366953448165288, 0.0032166379683476027, 0.49616858237547895)

In [83]:
aud_recursive_LSTM_RMSE, aud_recursive_LSTM_MAE, aud_recursive_LSTM_DA

(0.005716025064109846, 0.00420648458967849, 0.5067049808429118)

In [84]:
gbp_recursive_LSTM_RMSE, gbp_recursive_LSTM_MAE, gbp_recursive_LSTM_DA

(0.005707144618527106, 0.004224464636170401, 0.4827586206896552)

### Rolling LSTM pred

In [85]:
cad_rolling_LSTM_RMSE = root_mean_squared_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_MAE = mean_absolute_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_DA = direction_accuracy(cad_rolling_LSTM_pred, cad_groundtruth)
aud_rolling_LSTM_RMSE = root_mean_squared_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_MAE = mean_absolute_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_DA = direction_accuracy(aud_rolling_LSTM_pred, aud_groundtruth)
gbp_rolling_LSTM_RMSE = root_mean_squared_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_MAE = mean_absolute_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_DA = direction_accuracy(gbp_rolling_LSTM_pred, gbp_groundtruth)

In [86]:
cad_rolling_LSTM_RMSE, cad_rolling_LSTM_MAE, cad_rolling_LSTM_DA

(0.004366874890696316, 0.003216533494752228, 0.4942528735632184)

In [87]:
aud_rolling_LSTM_RMSE, aud_rolling_LSTM_MAE, aud_rolling_LSTM_DA

(0.00577983214485462, 0.0042638320321405545, 0.5067049808429118)

In [88]:
gbp_rolling_LSTM_RMSE, gbp_rolling_LSTM_MAE, gbp_rolling_LSTM_DA

(0.005705038932962677, 0.004221539465330905, 0.4827586206896552)

### Recursive Self-Attention pred

In [89]:
cad_recursive_Self_Attention_RMSE = root_mean_squared_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_MAE = mean_absolute_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_DA = direction_accuracy(cad_recursive_Self_Attention_pred, cad_groundtruth)
aud_recursive_Self_Attention_RMSE = root_mean_squared_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_MAE = mean_absolute_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_DA = direction_accuracy(aud_recursive_Self_Attention_pred, aud_groundtruth)
gbp_recursive_Self_Attention_RMSE = root_mean_squared_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_MAE = mean_absolute_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_DA = direction_accuracy(gbp_recursive_Self_Attention_pred, gbp_groundtruth)

In [90]:
cad_recursive_Self_Attention_RMSE, cad_recursive_Self_Attention_MAE, cad_recursive_Self_Attention_DA

(0.004372101752788658, 0.0032285911199098063, 0.4827586206896552)

In [91]:
aud_recursive_Self_Attention_RMSE, aud_recursive_Self_Attention_MAE, aud_recursive_Self_Attention_DA

(0.005902485725115945, 0.004439521080075167, 0.4779693486590038)

In [92]:
gbp_recursive_Self_Attention_RMSE, gbp_recursive_Self_Attention_MAE, gbp_recursive_Self_Attention_DA

(0.005711377556770118, 0.004226133230130188, 0.4827586206896552)

### Rolling Self-Attention pred

In [93]:
cad_rolling_Self_Attention_RMSE = root_mean_squared_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_MAE = mean_absolute_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_DA = direction_accuracy(cad_rolling_Self_Attention_pred, cad_groundtruth)
aud_rolling_Self_Attention_RMSE = root_mean_squared_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_MAE = mean_absolute_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_DA = direction_accuracy(aud_rolling_Self_Attention_pred, aud_groundtruth)
gbp_rolling_Self_Attention_RMSE = root_mean_squared_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_MAE = mean_absolute_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_DA = direction_accuracy(gbp_rolling_Self_Attention_pred, gbp_groundtruth)

In [94]:
cad_rolling_Self_Attention_RMSE, cad_rolling_Self_Attention_MAE, cad_rolling_Self_Attention_DA

(0.004372219300351518, 0.003228707155933871, 0.4827586206896552)

In [95]:
aud_rolling_Self_Attention_RMSE, aud_rolling_Self_Attention_MAE, aud_rolling_Self_Attention_DA

(0.005897562861361657, 0.004434635820803504, 0.4779693486590038)

In [96]:
gbp_rolling_Self_Attention_RMSE, gbp_rolling_Self_Attention_MAE, gbp_rolling_Self_Attention_DA

(0.005709054168491827, 0.004223106621892312, 0.48563218390804597)